### Inspect performance of iris interpolation schemes

In the Aerocom IDL tools, collocation of model data with point observations is done using neirest neighbour interpolation. The pyaerocom `GridData` class is based on (but not inherited from) the iris `Cube` class, which includes an interpolation method that takes one or multiple coordinates on input. The iris interpolation interface supports neirest neighbour and linear grid interpolation. 

This notebook was developed as a result of former tests, that revealed, that the `Cube` interpolation method can be considerably slow, since it loads the whole grid into memory (even if only a single point is accessed).

In [19]:
import numpy as np
import pyaerocom
import time
import iris

def load_model_data():
    data = pyaerocom.GridData()
    data._init_testdata_default()
    return data.grid

Let's start with running tests for extracting a time series at a single location using neirest neighbour interpolation. This is done in 3 ways:

1. Using the original Cube (global grid)
2. Using a Cube that is cropped around the point of interest before interpolation
3. Extracting directly using the closest indices

We use the following coordinates:

In [2]:
#single coordinate
lon, lat = 10, 10

#### Case 1 - Iris interface original grid

Using the iris interpolation interface on the original data grid stored in the `Cube`. Starting with loading the test datset.

In [38]:
%%time
cube = load_model_data()
s0_case1 = cube.interpolate(sample_points=[("longitude", lon), 
                                           ("latitude", lat)], scheme=iris.analysis.Nearest()).data

Rolling longitudes to -180 -> 180 definition
CPU times: user 941 ms, sys: 1.87 s, total: 2.81 s
Wall time: 13.2 s


This took quite a while, the reason for this is, that the Interpolator instance loads the whole grid into memory before doing the interpolation. So for a single point (or only a few points) this does not make a lot of sense. However, now that we have everything ready in memory, we can do the whole thing for a lot more points, without reloading the data, which is fast:

In [40]:
# whole globe in 1degree resolution
more_lons = np.arange(-180, 180, 1) #360 points
more_lats = np.arange(-90, 90, 1) #180 points

In [41]:
%time
sub = cube.interpolate(sample_points = [("longitude", more_lons), ("latitude", more_lats)], scheme=iris.analysis.Nearest())
print(sub.shape)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
(365, 180, 360)


Now, this gives us an instance of the `Cube` class. Let's just make sure, accessing the data does not take ages again:

In [34]:
%%time
tmp = sub.data

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 12.4 µs


#### Case 2 - Iris interface cropped grid

Using the iris interpolation interface after cropping the `Cube` within a suitable region-of-interest. For this, we first write a little helper function that determines a suitable crop interval.

In [35]:
def crop_around(lon, lat, stepdeg=2):
    lon_range = (lon-stepdeg, lon+stepdeg)
    lat_range = (lat-stepdeg, lat+stepdeg)
    return lon_range, lat_range

print(crop_around(lon, lat))

((102.21052631578948, 106.21052631578948), (2.7368421052631504, 6.73684210526315))


In [36]:
%%time
cube = load_model_data()
#get grid resolution
lonr, latr = crop_around(lon, lat)
cropped = cube.intersection(longitude=lonr, latitude=latr)
s0_case2 = cropped.interpolate(sample_points=[("longitude", lon), ("latitude", lat)], scheme=iris.analysis.Nearest()).data

Rolling longitudes to -180 -> 180 definition
CPU times: user 62.6 ms, sys: 208 ms, total: 271 ms
Wall time: 2.13 s


Well, this was considerably faster, but only got us one point (in about 2s). I spare you the time to loop over all points in the additional arrays. Make sure, the extracted arrays of both cases are equal.

In [9]:
np.testing.assert_array_equal(s0_case1, s0_case2)

#### Case 3 - Finding and extracting closest point using numpy

Let's load the data again and extract the lon / lat coordinate arrays

In [10]:
cube = load_model_data()
lons, lats = cube.coord("longitude").points, cube.coord("latitude").points
print(cube.shape)

Rolling longitudes to -180 -> 180 definition
(365, 451, 900)


Write a helper method that finds the current index

In [11]:
def get_closest_index(lons, lats, lon, lat):
    return (np.argmin(np.abs(lons - lon)), np.argmin(np.abs(lats - lat)))

And extract time series for the first point:

In [12]:
%%time
idx_lon, idx_lat = get_closest_index(lons, lats, lon, lat)
s0_case3 = cube[:, idx_lat, idx_lon].data

CPU times: user 40.7 ms, sys: 30.5 ms, total: 71.2 ms
Wall time: 376 ms


Again, before applying this method to all coordinates in ``more_lons, more_lats``, make sure the numbers are right.

In [13]:
np.testing.assert_array_equal(s0_case1, s0_case3)

Now for the whole thing:

In [14]:
%%time
data = np.empty((365, len(more_lats), len(more_lons)))
for i in range(len(more_lons)):
    for j in range(len(more_lats)):
        lon, lat = more_lons[i], more_lats[j]
        idx_lon, idx_lat = get_closest_index(lons, lats, lon, lat)
        data[:,j,i] = cube[:, idx_lat, idx_lon].data
        

KeyboardInterrupt: 

Now, this took more than 3 times as long as the case iris interface on the original grid. Note, however, that for a single or only a few points this method outperforms the iris method by far.

### Comparing Case 1 and Case 3 based on realistic obsdata case

Now comparing the two cases (i.e. iris, vs custom numpy method) based on the approximate number of stations of the AeroNet network (i.e. using 400 data points, i.e. a 20 x 20 point grid).

In [15]:
little_less_lons = np.linspace(-180, 180, 20)
little_less_lats = np.linspace(-90, 90, 20)

#### Case1 (400 datapoints)

In [16]:
%%time
cube = load_model_data()
cube.interpolate(sample_points = [("longitude", little_less_lons), 
                                             ("latitude", little_less_lats)], scheme=iris.analysis.Nearest()).data

Rolling longitudes to -180 -> 180 definition
CPU times: user 989 ms, sys: 1.79 s, total: 2.78 s
Wall time: 12.6 s


#### Case 3 (400 datapoints)

In [17]:
%%time
cube = load_model_data()
lons, lats = cube.coord("longitude").points, cube.coord("latitude").points

data = np.empty((365, len(little_less_lats), len(little_less_lons)))
for i in range(len(little_less_lons)):
    print(i)
    for j in range(len(little_less_lats)):
        lon, lat = little_less_lons[i], little_less_lats[j]
        idx_lon, idx_lat = get_closest_index(lons, lats, lon, lat)
        data[:,j,i] = cube[:, idx_lat, idx_lon].data

Rolling longitudes to -180 -> 180 definition
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: 

In [18]:
print(data.shape)

(365, 20, 20)
